# Frost heave compaction solver

This notebook solves a frost heave compaction problem in one spatial dimension (vertical). 

The code requires [FEniCSx](https://fenicsproject.org). The notebook can be run 
through a [Docker](https://www.docker.com) container with the command:

`docker run --init -ti -p 8888:8888 -v $(pwd):/home/fenics/shared -w /home/fenics/shared dolfinx/lab:stable`

Various imports:

In [ ]:
%load_ext autoreload
%autoreload 2
# add path to code
import sys
sys.path.insert(0, '../source')

In [ ]:
# !pip install --user ipyparallel

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipyparallel as ipp
import os 
from scipy.interpolate import interp1d
from scipy.optimize import root_scalar

Start ipyparallel cluster for running a bunch of examples in parallel

In [ ]:
mycluster = ipp.Cluster(n = 2)
rc = mycluster.start_and_connect_sync()
view = rc.load_balanced_view()
dview = rc[:]
dview.block = True
dview.execute('import numpy as np')
dview.execute('import sys')
dview.execute('sys.path.insert(0, "../source")')
dview.execute('from wrapper import wrapper_v')

Set initial conditions and timesteps for the examples in wrapper.py

Compute lens initiation condition for range of initial conditions at $v_i=0.037$:

In [ ]:
from wrapper import wrapper_v, z_0

In [ ]:
v_i = 0.037
mydict = dict(v_i = v_i)
dview.push(mydict);

wrapper_ = lambda i: wrapper_v(i,v_i)
parameters = list(range(z_0.size))
async_results = []
for i in parameters:
    async_result = view.apply_async(wrapper_, i)
    async_results.append(async_result)

rc.wait_interactive(async_results)

results = [ar.get() for ar in async_results]

fname = 'results-phase-37.npy'
res = {'results':results,'z_0':z_0,'timesteps':timesteps}
np.save(fname,res)

Repeat for three other examples: 

In [ ]:
v_i = 0.0375
mydict = dict(v_i = v_i)
dview.push(mydict);
wrapper_ = lambda i: wrapper_v(i,v_i)
parameters = list(range(z_0.size))
async_results = []
for i in parameters: 
    async_result = view.apply_async(wrapper_, i)
    async_results.append(async_result)

rc.wait_interactive(async_results)

results = [ar.get() for ar in async_results]

fname = 'results-phase-37p5.npy'
res = {'results':results,'z_0':z_0,'timesteps':timesteps}
np.save(fname,res)

In [ ]:
v_i = 0.038
mydict = dict(v_i = v_i)
dview.push(mydict);
wrapper_ = lambda i: wrapper_v(i,v_i)
parameters = list(range(z_0.size))
async_results = []
for i in parameters: 
    async_result = view.apply_async(wrapper_, i)
    async_results.append(async_result)

rc.wait_interactive(async_results)

results = [ar.get() for ar in async_results]

fname = 'results-phase-38.npy'
res = {'results':results,'z_0':z_0,'timesteps':timesteps}
np.save(fname,res)

In [ ]:
v_i = 0.0385
mydict = dict(v_i = v_i)
dview.push(mydict);
wrapper_ = lambda i: wrapper_v(i,v_i)
parameters = list(range(z_0.size))
async_results = []
for i in parameters: 
    async_result = view.apply_async(wrapper_, i)
    async_results.append(async_result)

rc.wait_interactive(async_results)

results = [ar.get() for ar in async_results]

fname = 'results-phase-38p5.npy'
res = {'results':results,'z_0':z_0,'timesteps':timesteps}
np.save(fname,res)

Plotting:

Interpolate recurrence relations:

In [ ]:
# WRITE LOAD SCRIPT!!!

In [ ]:
# zn_z1_c = zn_z1_c[z_0<12.1]
# zn_z2_c = zn_z2_c[z_0<12.1]
# zn_z3_c = zn_z3_c[z_0<12.1]
# zn_z4_c = zn_z4_c[z_0<12.1]
# z_0 = z_0[z_0<12.1]

In [ ]:
f_1 = interp1d(z_0,zn_z1_c,fill_value='extrapolate')
# f_2 = interp1d(z_0,zn_z2_c,fill_value='extrapolate')
# f_3 = interp1d(z_0,zn_z3_c,fill_value='extrapolate')
# f_4 = interp1d(z_0,zn_z4_c,fill_value='extrapolate')

Find fixed points:

In [ ]:
ze_1 = root_scalar(lambda z: f_1(z)-z,x0=8,x1=10,method='secant').root
# ze_2 = root_scalar(lambda z: f_2(z)-z,x0=8,x1=10,method='secant').root
# ze_3 = root_scalar(lambda z: f_3(z)-z,x0=8,x1=10,method='secant').root
# ze_4 = root_scalar(lambda z: f_4(z)-z,x0=8,x1=10,method='secant').root

In [ ]:
plt.figure(figsize=(8,6))
# plt.subplot(221)
# plt.title(r'(a)$\qquad v_\mathrm{i} = 0.037$',fontsize=20,loc='left')
# plt.plot(z_0,zn_z1_c,'o-',color='gray')
# plt.plot(z_0,z_0,':',color='k',alpha=0.25)
# plt.plot([limit1[2:j1-2]],[np.roll(limit1,-1)[2:j1-2]],'o',color='forestgreen',markersize=8,markeredgecolor='k')
# plt.plot(np.linspace(ze_1-1,ze_1+1),np.flipud(np.linspace(ze_1-1,ze_1+1)),'--',color='crimson',linewidth=2)
# plt.plot([ze_1],[ze_1],'s',color='crimson',markersize=7,markeredgecolor='k')
# plt.ylabel(r'$z_\mathrm{n}^{j+1}$',fontsize=20)  
# plt.gca().xaxis.set_ticklabels([])
# plt.yticks(fontsize=16)
# plt.xlim(4,12)
# plt.ylim(5,12.5)

# plt.subplot(222)
# plt.title(r'(b)$\qquad v_\mathrm{i} = 0.0375$',fontsize=20,loc='left')

# # # test cobweb
# #--------------------------
# N = 10
# x0 = limit3[5]
# x, y = x0, f_2(x0)
# for j in range(N):
#     fy = f_2(y)    
#     plt.plot([x, y], [y,  y], 'b', linewidth=1,alpha=0.25)
#     plt.plot([y, y], [y, fy], 'b', linewidth=1,alpha=0.25)
#     x, y = y, fy
# #--------------------------

# plt.plot(z_0,zn_z2_c,'o-',color='gray')
# plt.plot(z_0,z_0,':',color='k',alpha=0.25)
# plt.plot([limit3[2:j3-2]],[np.roll(limit3,-1)[2:j3-2]],'o',color='forestgreen',markersize=8,markeredgecolor='k')
# plt.plot(np.linspace(ze_2-1,ze_2+1),np.flipud(np.linspace(ze_2-1,ze_2+1)),'--',color='crimson',linewidth=2)
# plt.plot([ze_2],[ze_2],'s',color='crimson',markersize=7,markeredgecolor='k')




# plt.gca().yaxis.set_label_position("right")
# plt.gca().yaxis.tick_right()
# plt.ylabel(r'$z_\mathrm{n}^{j+1}$',fontsize=20)  
# plt.gca().xaxis.set_ticklabels([])
# plt.yticks(fontsize=16)
# plt.xlim(4,12)
# plt.ylim(5,12.5)

# plt.subplot(223)

# # # test cobweb
# #--------------------------
# N = 15
# x0 = limit2[5]
# x, y = x0, f_3(x0)
# for j in range(N):
#     fy = f_3(y)    
#     plt.plot([x, y], [y,  y], 'b', linewidth=1,alpha=0.25)
#     plt.plot([y, y], [y, fy], 'b', linewidth=1,alpha=0.25)
#     x, y = y, fy
# #--------------------------

# plt.title(r'(c)$\qquad v_\mathrm{i} = 0.038$',fontsize=20,loc='left')
# plt.plot(z_0,zn_z3_c,'o-',color='gray')
# plt.plot(z_0,z_0,':',color='k',alpha=0.25)
# plt.plot([limit2[2:j2-2]],[np.roll(limit2,-1)[2:j2-2]],'o',color='forestgreen',markersize=8,markeredgecolor='k')
# plt.plot(np.linspace(ze_3-1,ze_3+1),np.flipud(np.linspace(ze_3-1,ze_3+1)),'--',color='crimson',linewidth=2)
# plt.plot([ze_3],[ze_3],'s',color='crimson',markersize=7,markeredgecolor='k')


# plt.ylabel(r'$z_\mathrm{n}^{j+1}$',fontsize=20)  
# plt.xlabel(r'$z_\mathrm{n}^j$',fontsize=20)  
# plt.xticks(fontsize=16)
# plt.yticks(fontsize=16)
# plt.xlim(4,12)
# plt.ylim(5,12.5)

# plt.subplot(224)

# # test cobweb
#--------------------------
N = 20
q = 0
x0 = limit4[0]
x, y = x0, f_4(x0)
for j in range(N):
    fy = f_4(y)    
    plt.plot([x, y], [y,  y], 'b', linewidth=1,alpha=0.25)
    plt.plot([y, y], [y, fy], 'b', linewidth=1,alpha=0.25)
    x, y = y, fy
#--------------------------


plt.title(r'(d)$\qquad v_\mathrm{i} = 0.0385$',fontsize=20,loc='left')
plt.plot(z_0,zn_z4_c,'o-',color='gray')
plt.plot(z_0,z_0,':',color='k',alpha=0.25)
plt.plot(z_0,10+ze_4-z_0,':',color='k',alpha=0.25)
plt.plot(limit4[2:j4-2],np.roll(limit4,-1)[2:j4-2],'o',color='forestgreen',markersize=8,markeredgecolor='k',label='limit set')
plt.plot(np.linspace(ze_4-1,ze_4+1),np.flipud(np.linspace(ze_4-1,ze_4+1)),'--',color='crimson',linewidth=2)
plt.plot([ze_4],[ze_4],'s--',color='crimson',markersize=7,markeredgecolor='k',label='fixed point')

plt.gca().yaxis.set_label_position("right")
plt.gca().yaxis.tick_right()
plt.ylabel(r'$z_\mathrm{n}^{j+1}$',fontsize=20)  
plt.xlabel(r'$z_\mathrm{n}^j$',fontsize=20)  
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.legend(fontsize=14,ncol=2,loc=(-0.6,-0.35))
plt.xlim(4,12)
plt.ylim(5,12.5)
plt.savefig('maps',bbox_inches='tight')
plt.show()
plt.close()